
### Adoptionspeed_label - target value
0: adopted on the same day of listing 
    
1: adopted on the first week of listing
    
2: adopted on the first month of listing(7-30 days)
    
3: adopted on the 2nd and 3rd month of listing
    
4: stayed over 90 days of listing 
    

In [ ]:
# imports for pull data from database 
#import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.0.3'
#os.environ['SPARK_VERSION']=spark_version

# Install Spark, Java, Postgres
#!apt-get update
#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
#!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
#!pip install -q findspark
#!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Set Environment Variables
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
#import findspark
#findspark.init()

#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets  import data from database (skip line for now while database being put together) 
#from pyspark import SparkFiles
#url="https://<bucket name>.s3.(fillin here)"
#spark.sparkContext.addFile(url)
#pet_df = spark.read.csv(SparkFiles.get("cleaned_train.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
#pet_df.show()

In [20]:
# Initial imports.
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# load in csv (until database is up then detele this code)
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cleaned_train.csv to cleaned_train.csv
User uploaded file "cleaned_train.csv" with length 688147 bytes


In [3]:
# read csv into datafram to expore (until database is up then delete this code)
pet_df = pd.read_csv('cleaned_train.csv')
pet_df.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,VideoAmt,PhotoAmt,AdoptionSpeed,word_count,Fee_bins
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,0,1.0,2,69,2
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,2.0,0,23,0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,7.0,3,69,0
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,0,8.0,2,25,3
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,3.0,2,81,0


In [4]:
pet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14981 entries, 0 to 14980
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Type           14981 non-null  int64  
 1   Age            14981 non-null  int64  
 2   Breed1         14981 non-null  int64  
 3   Breed2         14981 non-null  int64  
 4   Gender         14981 non-null  int64  
 5   Color1         14981 non-null  int64  
 6   Color2         14981 non-null  int64  
 7   Color3         14981 non-null  int64  
 8   MaturitySize   14981 non-null  int64  
 9   FurLength      14981 non-null  int64  
 10  Vaccinated     14981 non-null  int64  
 11  Dewormed       14981 non-null  int64  
 12  Sterilized     14981 non-null  int64  
 13  Health         14981 non-null  int64  
 14  Quantity       14981 non-null  int64  
 15  VideoAmt       14981 non-null  int64  
 16  PhotoAmt       14981 non-null  float64
 17  AdoptionSpeed  14981 non-null  int64  
 18  word_c

# Split the Data into Training and Testing

In [6]:
# Create our features
X = pet_df.drop("AdoptionSpeed", axis=1)

# Create our target
y = pet_df["AdoptionSpeed"]

In [7]:
X.describe()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,VideoAmt,PhotoAmt,word_count,Fee_bins
count,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000,14981.000000
mean,1.457379,10.448301,265.270543,74.013083,1.776383,2.234430,3.221681,1.883052,1.861825,1.467459,1.731260,1.558774,1.914158,1.036513,1.525599,0.056805,3.749149,63.025165,0.377345
std,0.498197,18.150898,60.063415,123.012891,0.681575,1.745578,2.742033,2.984552,0.547721,0.599030,0.667581,0.695758,0.565922,0.199302,1.175047,0.346319,2.861154,69.306738,1.068104
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,1.000000,2.000000,265.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,2.000000,21.000000,0.000000
50%,1.000000,3.000000,266.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000,3.000000,44.000000,0.000000
75%,2.000000,11.000000,307.000000,179.000000,2.000000,3.000000,6.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,5.000000,81.000000,0.000000
max,2.000000,255.000000,307.000000,307.000000,3.000000,7.000000,7.000000,7.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,6.000000,8.000000,13.000000,1257.000000,5.000000


In [8]:
# Check the balance of our target values
y.value_counts()

4    4193
2    4032
3    3258
1    3088
0     410
Name: AdoptionSpeed, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({0: 311, 1: 2303, 2: 3034, 3: 2457, 4: 3130})

In [10]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## RandomForestClassifier

In [11]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=1) 

In [12]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [13]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions 

array([1, 4, 3, ..., 2, 2, 2])

In [14]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.41350774159103043

In [18]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)
cm
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1", "Actual 2","Actual 3", "Actual 4"], columns=["Predicted 0", "Predicted 1","Predicted 2","Predicted 3","Predicted 4"])

cm_df

,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 0,2,33,22,13,29
Actual 1,4,270,273,84,154
Actual 2,1,213,392,162,230
Actual 3,1,127,221,215,237
Actual 4,1,99,180,113,670


In [21]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 0,2,33,22,13,29
Actual 1,4,270,273,84,154
Actual 2,1,213,392,162,230
Actual 3,1,127,221,215,237
Actual 4,1,99,180,113,670


Accuracy Score : 0.41350774159103043
Classification Report
              precision    recall  f1-score   support

           0       0.22      0.02      0.04        99
           1       0.36      0.34      0.35       785
           2       0.36      0.39      0.38       998
           3       0.37      0.27      0.31       801
           4       0.51      0.63      0.56      1063

    accuracy                           0.41      3746
   macro avg       0.36      0.33      0.33      3746
weighted avg       0.40      0.41      0.40      3746



In [22]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.0090502 , 0.10963755, 0.07038043, 0.04714948, 0.03847839,
       0.06954524, 0.07102324, 0.03495131, 0.03736809, 0.0394702 ,
       0.02975295, 0.03174993, 0.02847046, 0.00910928, 0.03489444,
       0.01027246, 0.10998972, 0.18681237, 0.03189425])

In [23]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


[(0.18681237377150112, 'word_count'),
 (0.109989718847057, 'PhotoAmt'),
 (0.1096375481912168, 'Age'),
 (0.07102324470634117, 'Color2'),
 (0.07038042603872528, 'Breed1'),
 (0.06954523537473632, 'Color1'),
 (0.04714947730736535, 'Breed2'),
 (0.03947020420601331, 'FurLength'),
 (0.03847838738849204, 'Gender'),
 (0.03736809253306168, 'MaturitySize'),
 (0.034951310594454714, 'Color3'),
 (0.0348944438725393, 'Quantity'),
 (0.03189425025950297, 'Fee_bins'),
 (0.031749929446407935, 'Dewormed'),
 (0.029752953618307102, 'Vaccinated'),
 (0.028470457630819665, 'Sterilized'),
 (0.010272463142986228, 'VideoAmt'),
 (0.009109280747774512, 'Health'),
 (0.009050202322697464, 'Type')]